In [36]:
import numpy as np
import copy
import math

In [3]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])  #(m,n)
y_train = np.array([0, 0, 0, 1, 1, 1])                                           #(m,)

In [10]:
m,n = X_train.shape
w = np.zeros(n)
b = 3.0

In [15]:
def sigmoid (z):
    return 1/(1+np.exp(-z))

In [37]:
def logistic_cost (X_train,y_train,w,b):
    # cost = -1/m(ylogfwbxi+1-yilog1-fwbxi)
    cost = 0.0
    for i in range(m):
        z = np.dot(w,X_train[i]) + b
        f_wb_i = sigmoid(z)
        cost+= y_train[i] * np.log(f_wb_i) + (1 - y_train[i]) * np.log(1-f_wb_i)
    
    return -cost/m 




In [19]:
w_tmp = np.array([1,1])
b_tmp = -3
logistic_cost(X_train,y_train,w_tmp,b_tmp)

0.36686678640551745

In [31]:
def compute_gradient_logistic(X_train,y_train,w,b):
    m,n = X_train.shape
    dj_db = 0.0
    dj_dw = np.zeros(n)
    
    for i in range(m):
        z = np.dot(X_train[i],w) + b
        f_wb_i = sigmoid(z)
        err_i = f_wb_i - y_train[i]
        for j in range(n):
            dj_dw[j]+= err_i * X_train[i][j]
        dj_db+= err_i 
    return dj_db/m,dj_dw/m

In [32]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]


In [38]:
def gradient_descent(X_train,y_train,w_in,b_in,alpha,iterations):
    
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    for i in range(iterations):
        dj_db,dj_dw = compute_gradient_logistic(X_train,y_train,w,b)
        w = w - (alpha * dj_dw)
        b = b - (alpha * dj_db)
        
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( logistic_cost(X_train, y_train, w, b) )
        
        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(iterations / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
    
    return w,b

In [39]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

Iteration    0: Cost 0.684610468560574   
Iteration 1000: Cost 0.1590977666870456   
Iteration 2000: Cost 0.08460064176930081   
Iteration 3000: Cost 0.05705327279402531   
Iteration 4000: Cost 0.042907594216820076   
Iteration 5000: Cost 0.034338477298845684   
Iteration 6000: Cost 0.028603798022120097   
Iteration 7000: Cost 0.024501569608793   
Iteration 8000: Cost 0.02142370332569295   
Iteration 9000: Cost 0.019030137124109114   

updated parameters: w:[5.28123029 5.07815608], b:-14.222409982019837


### scikit-logistic-regression

In [40]:
X = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y = np.array([0, 0, 0, 1, 1, 1])

In [42]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X,y)

LogisticRegression()

### make a prediction scikit-learn

In [43]:
y_pred = lr_model.predict(X)

print("Prediction on training set:", y_pred)

Prediction on training set: [0 0 0 1 1 1]


### Calculate accuracy

You can calculate this accuracy of this model by calling the `score` function.

In [44]:
print("Accuracy on training set:", lr_model.score(X, y))

Accuracy on training set: 1.0


## Regularized cost and gradient descent for linear and logistic regression

In [54]:
def compute_cost_linear_reg(X,y,w,b,lam_=1):
    m,n = X.shape
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i],w) + b
        cost += (f_wb_i - y[i]) ** 2
    cost = cost / (2 * m)

    reg_cost = 0.0
    for j in range(n):
        reg_cost += (w[j] ** 2)
    reg_cost = (lam_/(2*m)) * reg_cost

    return cost + reg_cost


In [56]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Regularized cost:", cost_tmp)

Regularized cost: 0.07917239320214275


In [58]:
def compute_cost_logistic_reg(X,y,w,b,lam_=1):
    m,n = X.shape
    cost = 0.0

    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost += - (y[i]*np.log(f_wb_i) + (1-y[i])*np.log(1-f_wb_i))
    
    cost = cost/m

    reg_cost = 0.0

    for j in range(n):
        reg_cost+=(w[j]**2)
    reg_cost = (lam_/(2*m)) * reg_cost

    return reg_cost + cost



In [59]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print("Regularized cost:", cost_tmp)

Regularized cost: 0.6850849138741673


In [64]:
def compute_gradient_linear_reg(X, y, w, b, lambda_):
    m,n = X.shape
    dj_dw = np.zeros(n)
    dj_db = 0.0

    for i in range(m):
        f_wb_i = np.dot(X[i],w) + b
        error_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] += error_i * X[i,j]
        dj_db += error_i
    dj_dw = dj_dw/m
    dj_db = dj_db/m

    for j in range(n):
        dj_dw[j] += (lambda_/m) * w[j]
    
    return dj_db,dj_dw


In [65]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

dj_db: 0.6648774569425726
Regularized dj_dw:
 [0.29653214748822276, 0.4911679625918033, 0.21645877535865857]


In [66]:
def compute_gradient_logistic_reg(X, y, w, b, lambda_): 
    m,n = X.shape
    dj_dw = np.zeros(n)
    dj_db = 0.0
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] += err_i * X[i,j]
        dj_db += err_i
    
    dj_dw = dj_dw/m
    dj_db = dj_db/m

    for j in range(n):
        dj_dw[j] += (lambda_/m) * w[j]

    return dj_db,dj_dw

In [67]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp =  compute_gradient_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

dj_db: 0.341798994972791
Regularized dj_dw:
 [0.17380012933994293, 0.32007507881566943, 0.10776313396851499]
